In [1]:
import pandas as pd
import numpy as np

---

## Driver Data

In [2]:
driver_data_path = "./data/driver_initial.csv"
driver_data = pd.read_csv(driver_data_path, engine="pyarrow")
# driver_data

## Order Data

In [3]:
order_data_path = "./data/order.csv"
order_data = pd.read_csv(order_data_path, engine="pyarrow")
# order_data

## Order-Driver Data

In [4]:
order_driver_data_path = "./data/order_driver.csv"
order_driver_data = pd.read_csv(order_driver_data_path, engine="pyarrow")
# order_driver_data

---

## Distance Validation Data

In [5]:
dist_val_data = order_driver_data[
    [
        "id",
        "order_id",
        "driver_id",
        "driver_distance",
        "distance",
        "accept",
        "driver_lat",
        "driver_lon",
    ]
]
# dist_val_data[dist_val_data["driver_id"] == 8]
# dist_val_data

In [6]:
dist_val_data = pd.merge(
    dist_val_data,
    order_data[["order_id", "pickup_lat", "pickup_lon", "dropoff_lat", "dropoff_lon"]],
    on="order_id",
    how="left",
)

# dist_val_data

In [7]:
dist_val_data["driver_to_pickup_distance"] = (
    np.sqrt(
        (dist_val_data["driver_lat"] - dist_val_data["pickup_lat"]) ** 2
        + (dist_val_data["driver_lon"] - dist_val_data["pickup_lon"]) ** 2
    )
    * 111
    * 1000
)

# Calculate pickup to dropoff distance
dist_val_data["pickup_to_dropoff_distance"] = (
    np.sqrt(
        (dist_val_data["pickup_lat"] - dist_val_data["dropoff_lat"]) ** 2
        + (dist_val_data["pickup_lon"] - dist_val_data["dropoff_lon"]) ** 2
    )
    * 111
    * 1000
)

dist_val_data["driver_to_dropoff_distance"] = (
    np.sqrt(
        (dist_val_data["driver_lat"] - dist_val_data["dropoff_lat"]) ** 2
        + (dist_val_data["driver_lon"] - dist_val_data["dropoff_lon"]) ** 2
    )
    * 111
    * 1000
)

dist_val_data["total_driver_trip_distance"] = (
    dist_val_data["driver_to_pickup_distance"]
    + dist_val_data["pickup_to_dropoff_distance"]
)

In [8]:
dist_val_data[
    [
        "id",
        "order_id",
        "driver_id",
        "driver_distance",
        "driver_to_pickup_distance",
        "distance",
        "driver_to_dropoff_distance",
        "total_driver_trip_distance",
    ]
]

,id,order_id,driver_id,driver_distance,driver_to_pickup_distance,distance,driver_to_dropoff_distance,total_driver_trip_distance
0,33510101,4863457,8,1662,3185.924389,7000,4915.466903,7364.220956
1,33510106,4863457,8,1667,3186.566947,7000,4907.993360,7364.863513
2,33510116,4863457,8,1680,3206.284556,7000,4921.752143,7384.581123
3,33510120,4863459,8,1895,NaN,9000,NaN,NaN
4,33510123,4863457,8,1667,3189.364763,7000,4913.147326,7367.661330
...,...,...,...,...,...,...,...,...
336539,33846028,4890389,21730,2745,2042.519728,5000,6336.401680,6606.433343
336540,33846064,4890389,21730,2414,1903.526190,5000,6082.082496,6467.439804
336541,33846089,4890396,21730,1422,968.500420,7000,6154.010258,6597.880826
336542,33846126,4890400,21730,2549,2430.477163,10000,2410.311940,2451.071214


In [9]:
order_data.columns

Index(['order_id', 'driver_id', 'customer_id', 'status', 'user_rate',
       'driver_rate', 'datetime', 'updated_at', 'driver_commission', 'route',
       'driver_delivered', 'driver_seen', 'tries', 'passenger_feedback',
       'driver_message', 'receipt_id', 'voice', 'return_to_id',
       'payment_at_id', 'submitted_by_id', 'accepted_at', 'started_at',
       'completed_at', 'unique_id', 'link_seen', 'service_type_id', 'source',
       'created_at_jd', 'created_at_jm', 'created_at_jy', 'sent_to_all',
       'driver_photo', 'last_broadcast', 'accept_lat', 'accept_lng',
       'auto_resend_until', 'is_messaging_enabled', 'has_walking', 'is_hurry',
       'order_tags', 'called_number', 'overload_photo',
       'has_reserved_discount', 'weather', 'temperature', 'is_very_hurry',
       'receipt_photo', 'is_future_order', 'sub_return_by', 'public_id',
       'accept', 'customer_price', 'distance', 'payment', 'commissionPercent',
       'hour', 'weekday', 'complete_time', 'wait_accept_sec',

In [10]:
order_data[["order_id", "driver_id", "accepted_at", "started_at", "completed_at"]]

,order_id,driver_id,accepted_at,started_at,completed_at
0,4863451,NaN,NaT,NaT,NaT
1,4863452,21137.0,2025-04-07 08:12:27,2025-04-07 08:31:52,2025-04-07 09:03:28
2,4863453,21577.0,2025-04-07 08:11:38,2025-04-07 08:21:40,2025-04-07 08:42:56
3,4863454,1599.0,2025-04-07 08:33:13,2025-04-07 08:33:29,2025-04-07 09:15:27
4,4863455,20416.0,2025-04-07 08:23:02,2025-04-07 08:37:41,2025-04-07 10:03:23
...,...,...,...,...,...
27001,4890452,NaN,NaT,NaT,NaT
27002,4890453,NaN,NaT,NaT,NaT
27003,4890454,2443.0,2025-04-21 23:23:24,2025-04-21 23:23:41,2025-04-22 22:41:59
27004,4890455,14432.0,2025-04-21 23:54:16,2025-04-21 23:54:23,NaT


---

## Accept more than 1 orders

In [11]:
# Ensure 'started_at' and 'completed_at' columns are datetime objects
order_data["started_at"] = pd.to_datetime(order_data["started_at"])
order_data["completed_at"] = pd.to_datetime(order_data["completed_at"])

# Sort data by driver_id and then by started_at to facilitate checking for overlaps
sorted_orders = order_data.sort_values(by=["driver_id", "started_at"])

# List to store all detected overlapping order pairs
overlapping_orders_list = []

# Group by driver_id and iterate through each driver's orders
for driver_id, driver_df in sorted_orders.groupby("driver_id"):
    # Iterate through consecutive orders for the current driver
    for i in range(len(driver_df) - 1):
        current_order = driver_df.iloc[i]
        next_order = driver_df.iloc[i + 1]

        # Check if the next order's start time is before the current order's completion time
        # This indicates an overlap in their active delivery periods
        if next_order["started_at"] < current_order["completed_at"]:
            overlapping_orders_list.append(
                {
                    "driver_id": driver_id,
                    "order_1_id": current_order["order_id"],
                    "order_1_started_at": current_order["started_at"],
                    "order_1_completed_at": current_order["completed_at"],
                    "order_2_id": next_order["order_id"],
                    "order_2_started_at": next_order["started_at"],
                    "order_2_completed_at": next_order["completed_at"],
                }
            )

# Convert the list of overlapping orders into a DataFrame
overlapping_df = pd.DataFrame(overlapping_orders_list)

if not overlapping_df.empty:
    print("Drivers with overlapping order completion times:")
    # print(overlapping_df)
    overlapping_ids = overlapping_df["driver_id"].astype(int).unique().tolist()
    print(f"The IDs of drivers with overlapping orders are: {overlapping_ids}")
else:
    print("No drivers found completing multiple orders in overlapping time periods.")


overlapping_df

Drivers with overlapping order completion times:
The IDs of drivers with overlapping orders are: [1691, 2101, 10113, 13843, 13963]


,driver_id,order_1_id,order_1_started_at,order_1_completed_at,order_2_id,order_2_started_at,order_2_completed_at
0,1691.0,4874800,2025-04-13 17:19:44,2025-04-13 17:31:58,4874817,2025-04-13 17:26:44,NaT
1,2101.0,4886905,2025-04-20 12:48:04,2025-04-20 15:11:55,4887308,2025-04-20 13:56:03,NaT
2,10113.0,4885266,2025-04-19 15:53:50,2025-04-19 16:11:28,4885279,2025-04-19 16:06:02,NaT
3,13843.0,4872230,2025-04-12 13:31:58,2025-04-12 14:20:44,4872219,2025-04-12 13:43:07,2025-04-12 14:27:59
4,13843.0,4873627,2025-04-13 11:20:44,2025-04-13 11:46:18,4873643,2025-04-13 11:26:27,2025-04-13 11:36:05
5,13843.0,4873826,2025-04-13 11:47:28,2025-04-13 12:09:19,4873838,2025-04-13 11:59:05,2025-04-13 13:04:31
6,13843.0,4876546,2025-04-14 13:57:19,2025-04-14 15:09:19,4876483,2025-04-14 14:06:57,2025-04-14 14:46:14
7,13843.0,4880359,2025-04-16 12:24:47,2025-04-16 12:57:01,4880515,2025-04-16 12:55:25,2025-04-16 14:10:59
8,13843.0,4886212,2025-04-20 09:56:38,2025-04-20 10:36:45,4886265,2025-04-20 10:17:08,2025-04-20 10:54:14
9,13843.0,4886577,2025-04-20 11:07:12,2025-04-20 13:04:18,4886512,2025-04-20 11:20:42,2025-04-20 12:06:58


In [12]:
order_data.shape

(27006, 71)

---

## Driver Update Data

In [13]:
driver_update2_data_path = './data/driver_update2.csv'
driver_update2_data = pd.read_csv(driver_update2_data_path)
driver_update2_data

,order_id,driver_id,driver_lat,driver_lon,driver_area,datetime,work_time_minutes,working
0,4863457,8,32.713852,51.631945,621,4/7/2025 8:20,0.000000,0
1,4863457,8,32.713770,51.631928,621,4/7/2025 8:20,0.150000,1
2,4863457,8,32.713827,51.631757,621,4/7/2025 8:24,4.116667,1
3,4863459,8,32.713812,51.631908,621,4/7/2025 8:27,7.100000,1
4,4863457,8,32.713812,51.631908,621,4/7/2025 8:27,7.200000,1
...,...,...,...,...,...,...,...,...
336539,4890389,21730,32.664392,51.717634,445,4/21/2025 20:27,77.383333,1
336540,4890389,21730,32.661866,51.716383,445,4/21/2025 20:28,78.350000,1
336541,4890396,21730,32.661405,51.710233,444,4/21/2025 20:30,80.583333,1
336542,4890400,21730,32.655327,51.722866,445,4/21/2025 20:35,85.166667,1


In [16]:
len(driver_update2_data['driver_area'].unique().tolist())

501